# Produce input catalogs for lensed system postage stamp generation

_Last Updated: Bryce Kalmbach, October 2018_

This is a notebook to produce the host AGN and SN catalogs that are used to created FITS postage stamps of the lensed systems.

## Load in galaxy catalogs as dataframes

These come from unsprinkled instance catalogs of the DDF region.

In [2]:
import pandas as pd
from astropy.io import fits
import numpy as np
from desc.sims.GCRCatSimInterface import InstanceCatalogWriter
from lsst.sims.utils import SpecMap
import matplotlib.pyplot as plt
from lsst.utils import getPackageDir
from lsst.sims.photUtils import Sed, BandpassDict, Bandpass
from lsst.sims.catUtils.matchSED import matchBase
import os
%matplotlib inline

/global/common/software/lsst/common/miniconda/py3-4.3.21-env/lib/python3.6/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
/global/homes/b/brycek/DC2/sims_catalogs/python/lsst/sims/catalogs/db/dbConnection.py:553: UserWarning: Duplicate object type id 4 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')
/global/homes/b/brycek/DC2/sims_catalogs/python/lsst/sims/catalogs/db/dbConnection.py:553: UserWarning: Duplicate object type id 28 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')


In [3]:
base_columns = ['prefix', 'uniqueId', 'raPhoSim', 'decPhoSim',
                        'phosimMagNorm', 'sedFilepath', 'redshift',
                        'shear1', 'shear2', 'kappa', 'raOffset', 'decOffset',
                        'spatialmodel']

In [4]:
df_galaxy = pd.read_csv(os.path.join(os.environ['SCRATCH'],
                                     'bulge_gal_cat_197356.txt.gz'),
                                delimiter=' ', header=None,
                                names=base_columns+['majorAxis', 'minorAxis',
                                                    'positionAngle', 'sindex',
                                                    'internalExtinctionModel',
                                                    'internalAv', 'internalRv',
                                                    'galacticExtinctionModel',
                                                    'galacticAv', 'galacticRv'])

In [5]:
df_disk = pd.read_csv(os.path.join(os.environ['SCRATCH'],
                                     'disk_gal_cat_197356.txt.gz'),
                                delimiter=' ', header=None,
                                names=base_columns+['majorAxis', 'minorAxis',
                                                    'positionAngle', 'sindex',
                                                    'internalExtinctionModel',
                                                    'internalAv', 'internalRv',
                                                    'galacticExtinctionModel',
                                                    'galacticAv', 'galacticRv'])

In [6]:
df_agn = pd.read_csv(os.path.join(os.environ['SCRATCH'],
                                     'agn_gal_cat_197356.txt.gz'),
                                 delimiter=' ', header=None,
                                 names=base_columns+['internalExtinctionModel',
                                                     'galacticExtinctionModel',
                                                     'galacticAv', 'galacticRv'])

In [7]:
df_agn['galaxy_id'] = np.right_shift(df_agn['uniqueId'], 10)

In [8]:
df_agn.head()

,prefix,uniqueId,raPhoSim,decPhoSim,phosimMagNorm,sedFilepath,redshift,shear1,shear2,kappa,raOffset,decOffset,spatialmodel,internalExtinctionModel,galacticExtinctionModel,galacticAv,galacticRv,galaxy_id
0,object,9216001276021,53.435559,-26.945783,25.086426,agnSED/agn.spec.gz,0.047209,-0.000021,-0.000006,-0.000045,0,0,point,none,CCM,0.027765,3.1,9000001246
1,object,9216001381493,53.736954,-26.539365,22.238127,agnSED/agn.spec.gz,0.047179,-0.000004,-0.000004,-0.000018,0,0,point,none,CCM,0.035462,3.1,9000001349
2,object,9216001382517,53.765088,-26.474307,21.005609,agnSED/agn.spec.gz,0.047179,-0.000023,-0.000011,-0.000016,0,0,point,none,CCM,0.033978,3.1,9000001350
3,object,9216001385589,53.721399,-26.534063,25.878225,agnSED/agn.spec.gz,0.047179,-0.000004,-0.000002,-0.000025,0,0,point,none,CCM,0.036798,3.1,9000001353
4,object,9216001911925,53.839947,-26.832082,23.400420,agnSED/agn.spec.gz,0.041929,-0.000023,-0.000004,-0.000053,0,0,point,none,CCM,0.035429,3.1,9000001867


In [9]:
df_galaxy['galaxy_id'] = np.right_shift(df_galaxy['uniqueId'], 10)

In [10]:
df_galaxy.head()

,prefix,uniqueId,raPhoSim,decPhoSim,phosimMagNorm,sedFilepath,redshift,shear1,shear2,kappa,...,minorAxis,positionAngle,sindex,internalExtinctionModel,internalAv,internalRv,galacticExtinctionModel,galacticAv,galacticRv,galaxy_id
0,object,9216000199777,53.545161,-27.085175,22.160876,galaxySED/Inst.16E09.0005Z.spec.gz,0.021692,-0.000000e+00,-0.000000,0.000000,...,0.385211,71.035500,4,CCM,-0.000000,1.000000,CCM,0.034481,3.1,9000000195
1,object,9216000809057,53.524564,-26.777660,21.390245,galaxySED/Burst.32E09.02Z.spec.gz,0.047217,6.460591e-07,0.000002,-0.000014,...,1.381355,88.921657,4,CCM,0.028208,0.470165,CCM,0.027570,3.1,9000000790
2,object,9216000943201,53.100701,-26.408293,23.306178,galaxySED/Burst.50E09.02Z.spec.gz,0.047701,1.951021e-05,0.000013,-0.000030,...,0.174265,123.970378,4,CCM,0.023759,0.415067,CCM,0.029433,3.1,9000000921
3,object,9216001003617,53.747821,-26.303476,19.648862,galaxySED/Burst.12E10.0005Z.spec.gz,0.041702,-1.047741e-05,0.000014,-0.000029,...,0.613943,175.763044,4,CCM,0.023664,0.491308,CCM,0.033555,3.1,9000000980
4,object,9216001114209,54.129535,-26.547473,18.680661,galaxySED/Exp.80E09.04Z.spec.gz,0.034863,-1.966811e-05,-0.000008,-0.000043,...,0.598062,88.153042,4,CCM,0.034661,0.500124,CCM,0.037492,3.1,9000001088


In [11]:
df_disk['galaxy_id'] = np.right_shift(df_disk['uniqueId'], 10)

In [12]:
df_disk.head()

,prefix,uniqueId,raPhoSim,decPhoSim,phosimMagNorm,sedFilepath,redshift,shear1,shear2,kappa,...,minorAxis,positionAngle,sindex,internalExtinctionModel,internalAv,internalRv,galacticExtinctionModel,galacticAv,galacticRv,galaxy_id
0,object,9216000184427,53.296930,-27.053911,21.555392,galaxySED/Exp.62E09.002Z.spec.gz,0.022239,-0.0,-0.0,0.0,...,0.456166,16.516023,1,CCM,0.095838,0.435907,CCM,0.029552,3.1,9000000180
1,object,9216000199787,53.545161,-27.085175,27.990970,galaxySED/Exp.50E08.04Z.spec.gz,0.021692,-0.0,-0.0,0.0,...,0.741451,71.035500,1,CCM,0.008377,0.431099,CCM,0.034481,3.1,9000000195
2,object,9216000280683,53.078152,-26.627776,21.921702,galaxySED/Inst.12E09.0005Z.spec.gz,0.022558,-0.0,-0.0,0.0,...,0.494495,92.899542,1,CCM,0.007408,0.335971,CCM,0.030726,3.1,9000000274
3,object,9216000287851,53.521768,-26.715249,20.581922,galaxySED/Exp.62E09.002Z.spec.gz,0.014010,-0.0,-0.0,0.0,...,0.974957,3.872094,1,CCM,0.129536,0.436745,CCM,0.028743,3.1,9000000281
4,object,9216000467051,53.667151,-26.774928,22.032053,galaxySED/Exp.50E09.0005Z.spec.gz,0.015892,-0.0,-0.0,0.0,...,1.108397,154.836313,1,CCM,0.101845,0.433304,CCM,0.035414,3.1,9000000456


## Make the AGN catalogs

What we are going to do is take the information on the unlensed bulge and disks from the instance catalog above and combine it with information on the lens that will be sprinkled in front of it so that we can accurately model the appearance of the lensed hosts in a fits postage stamp to include with the instance catalogs and give us properly modeled host galaxy images. We first do it for the AGN here and then repeat for the SNe below.

### Load in the AGN lensed systems and cache file

The cache file associates the `galaxy_id` values with the `twinklesId` values in the lensed system catalog. The file `cosmoDC2_v1.0_matched_AGN.fits` contains the information on the lensed systems.

In [15]:
hdulist = fits.open('../../data/cosmoDC2_v1.0_matched_AGN.fits')
twinkles_lenses = hdulist[1].data

In [16]:
agn_cache_df = pd.read_csv('../../data/cosmoDC2_v1.0_agn_cache.csv')

Next we add in the lens properties to go with each of the images.

In [80]:
#TwinklesId ranges from 0 - 1040
ra_images = []
dec_images = []
num_images = []
twinkles_system_images = []
unique_id_lens = []

unique_id_bulge = []
unique_id_disk = []

bulge_index = []
disk_index = []

disk_valid = []

i = 0
for twinkles_system in range(1041):
    
    if twinkles_system % 100 == 0:
        print(twinkles_system)
    
    cache_info = agn_cache_df.query('twinkles_system == %i' % twinkles_system)
    gal_id = cache_info['galtileid'].values[0]
    lens_info = twinkles_lenses[np.where(twinkles_lenses['twinklesId'] == twinkles_system)]
    bulge_info = df_galaxy.query('galaxy_id == %i' % gal_id)
    disk_info = df_disk.query('galaxy_id == %i' % gal_id)

    
    #print(bulge_info, disk_info)
    
    for lens_image in range(lens_info[0]['NIMG']):
        ra_shift = np.radians(lens_info[0]['XIMG'][lens_image] / 3600.) / np.cos(np.radians(bulge_info['decPhoSim']))
        dec_shift = np.radians(lens_info[0]['YIMG'][lens_image] / 3600.)
        
        image_ra = np.radians(bulge_info['raPhoSim']) + ra_shift
        image_dec = np.radians(bulge_info['decPhoSim'] + dec_shift)
        
        ra_images.append(np.degrees(image_ra).values[0])
        dec_images.append(np.degrees(image_dec).values[0])
        num_images.append(lens_image)
        unique_id_lens.append(bulge_info['uniqueId'].values[0])
        twinkles_system_images.append(twinkles_system)
        base_id = (gal_id + int(1.5e10))*10000 + twinkles_system*4 + lens_image
        unique_id_bulge.append(np.left_shift(base_id, 10) + 97)
        unique_id_disk.append(np.left_shift(base_id, 10) + 107)
        
        bulge_index.append(bulge_info.index[0])
        
        if len(disk_info) == 0:
            print(twinkles_system)
        else:
            disk_index.append(disk_info.index[0])
            disk_valid.append(i)
            
        i += 1

0
100
200
300
400
500
600
700
740
740
740
740
800
900
1000


In [98]:
host_bulge = df_galaxy.iloc[bulge_index]

In [99]:
host_bulge = host_bulge.reset_index(drop=True)

In [100]:
host_bulge['twinkles_system'] = twinkles_system_images

In [101]:
host_bulge['image_number'] = num_images

In [102]:
host_bulge['uniqueId_lens'] = unique_id_lens

In [103]:
host_bulge['uniqueId'] = unique_id_bulge

In [104]:
host_bulge['raPhoSim_lens'] = host_bulge['raPhoSim']
host_bulge['decPhoSim_lens'] = host_bulge['decPhoSim']

In [105]:
host_bulge['raPhoSim'] = ra_images
host_bulge['decPhoSim'] = dec_images

In [106]:
host_bulge.head()

,prefix,uniqueId,raPhoSim,decPhoSim,phosimMagNorm,sedFilepath,redshift,shear1,shear2,kappa,...,internalRv,galacticExtinctionModel,galacticAv,galacticRv,galaxy_id,twinkles_system,image_number,uniqueId_lens,raPhoSim_lens,decPhoSim_lens
0,object,164133666611200097,53.217696,-27.890644,21.392373,galaxySED/Burst.16E09.0005Z.spec.gz,0.960092,-0.003274,-0.003062,0.003915,...,0.44442,CCM,0.023508,3.1,1028678380,0,0,1053366661217,53.217660,-27.890644
1,object,164133666611201121,53.217523,-27.890642,21.392373,galaxySED/Burst.16E09.0005Z.spec.gz,0.960092,-0.003274,-0.003062,0.003915,...,0.44442,CCM,0.023508,3.1,1028678380,0,1,1053366661217,53.217660,-27.890644
2,object,164997824153604193,53.641069,-28.543523,23.878532,galaxySED/Burst.16E09.25Z.spec.gz,1.793886,0.000929,-0.009666,0.000049,...,1.00000,CCM,0.024226,3.1,1113068765,1,0,1139782415457,53.640802,-28.543518
3,object,164997824153605217,53.640522,-28.543519,23.878532,galaxySED/Burst.16E09.25Z.spec.gz,1.793886,0.000929,-0.009666,0.000049,...,1.00000,CCM,0.024226,3.1,1113068765,1,1,1139782415457,53.640802,-28.543518
4,object,166179311390728289,53.192471,-27.816045,23.508753,galaxySED/Exp.32E09.02Z.spec.gz,2.251758,0.006626,-0.000344,-0.034656,...,1.00000,CCM,0.023387,3.1,1228448378,2,0,1257931139169,53.192584,-27.816053


In [107]:
host_disk = df_disk.iloc[disk_index]

In [108]:
host_disk = host_disk.reset_index(drop=True)

In [110]:
host_disk['twinkles_system'] = np.array(twinkles_system_images)[disk_valid]

In [111]:
host_disk['image_number'] = np.array(num_images)[disk_valid]

In [112]:
host_disk['uniqueId_lens'] = np.array(unique_id_lens)[disk_valid]

In [113]:
host_disk['uniqueId'] = np.array(unique_id_disk)[disk_valid]

In [114]:
host_disk['raPhoSim_lens'] = host_disk['raPhoSim']
host_disk['decPhoSim_lens'] = host_disk['decPhoSim']

In [115]:
host_disk['raPhoSim'] = np.array(ra_images)[disk_valid]
host_disk['decPhoSim'] = np.array(dec_images)[disk_valid]

In [116]:
host_disk.head()

,prefix,uniqueId,raPhoSim,decPhoSim,phosimMagNorm,sedFilepath,redshift,shear1,shear2,kappa,...,internalRv,galacticExtinctionModel,galacticAv,galacticRv,galaxy_id,twinkles_system,image_number,uniqueId_lens,raPhoSim_lens,decPhoSim_lens
0,object,164133666611200107,53.217696,-27.890644,29.084400,galaxySED/Exp.20E08.04Z.spec.gz,0.960092,-0.003274,-0.003062,0.003915,...,0.442226,CCM,0.023508,3.1,1028678380,0,0,1053366661217,53.217660,-27.890644
1,object,164133666611201131,53.217523,-27.890642,29.084400,galaxySED/Exp.20E08.04Z.spec.gz,0.960092,-0.003274,-0.003062,0.003915,...,0.442226,CCM,0.023508,3.1,1028678380,0,1,1053366661217,53.217660,-27.890644
2,object,164997824153604203,53.641069,-28.543523,45.262988,galaxySED/Exp.40E08.04Z.spec.gz,1.793886,0.000929,-0.009666,0.000049,...,0.666667,CCM,0.024226,3.1,1113068765,1,0,1139782415457,53.640802,-28.543518
3,object,164997824153605227,53.640522,-28.543519,45.262988,galaxySED/Exp.40E08.04Z.spec.gz,1.793886,0.000929,-0.009666,0.000049,...,0.666667,CCM,0.024226,3.1,1113068765,1,1,1139782415457,53.640802,-28.543518
4,object,166179311390728299,53.192471,-27.816045,53.362623,galaxySED/Inst.25E07.002Z.spec.gz,2.251758,0.006626,-0.000344,-0.034656,...,1.000000,CCM,0.023387,3.1,1228448378,2,0,1257931139169,53.192584,-27.816053


In [119]:
host_bulge.to_csv('../../data/cosmoDC2_v1.0_bulge_agn_host.csv')
host_disk.to_csv('../../data/cosmoDC2_v1.0_disk_agn_host.csv')

## Make the SNe catalogs

We do the same as we did for the lensed AGN systems with the lensed SNe systems below.

### Load in the SNE lensed systems and cache file

The cache file associates the `galaxy_id` values with the `twinklesId` values in the lensed system catalog. The file `cosmoDC2_v1.0_sne_cat.csv` contains the information on the images.

In [13]:
sne_df = pd.read_csv('../../data/cosmoDC2_v1.0_sne_cat.csv')

In [14]:
sne_cache_df = pd.read_csv('../../data/cosmoDC2_v1.0_sne_cache.csv')

In [18]:
sne_df.head()

,t0,sigma,gamma,e,theta_e,theta_gamma,zs,zl,snx,sny,...,c,lensgal_magnorm,lensgal_sed,twinkles_sysno,td,x,y,mu,imno,t_start
0,60525.724354,224.378094,0.052864,0.094133,223.912553,227.312450,0.178801,0.081910,0.212492,-0.210146,...,0.0,17.019237,galaxySED/Burst.10E10.04Z.spec.gz,1100,0.000000,0.701332,-0.717703,6.763210,0,60525.724354
1,60525.724354,224.378094,0.052864,0.094133,223.912553,227.312450,0.178801,0.081910,0.212492,-0.210146,...,0.0,17.019237,galaxySED/Burst.10E10.04Z.spec.gz,1100,8.066515,-0.309977,0.306061,-1.042185,1,60533.790868
2,61020.718041,217.298401,0.119059,0.718813,114.731718,341.413819,0.243307,0.162508,0.048668,0.125921,...,0.0,18.782605,galaxySED/Inst.80E09.04Z.spec.gz,1101,0.000000,-0.178645,0.520265,1.807370,0,61020.718041
3,61020.718041,217.298401,0.119059,0.718813,114.731718,341.413819,0.243307,0.162508,0.048668,0.125921,...,0.0,18.782605,galaxySED/Inst.80E09.04Z.spec.gz,1101,4.535220,0.390857,0.126209,-1.260907,1,61025.253261
4,61020.718041,217.298401,0.119059,0.718813,114.731718,341.413819,0.243307,0.162508,0.048668,0.125921,...,0.0,18.782605,galaxySED/Inst.80E09.04Z.spec.gz,1101,8.273781,-0.157020,-0.116978,-0.389962,2,61028.991822


Next we add in the lens properties to go with each of the images.

In [33]:
#TwinklesId ranges from 1100 - 2077
ra_images = []
dec_images = []
num_images = []
twinkles_system_images = []
unique_id_lens = []

unique_id_bulge = []
unique_id_disk = []

bulge_index = []
disk_index = []

disk_valid = []

i = 0
for twinkles_system in range(1100, 2078):
    
    if twinkles_system % 100 == 0:
        print(twinkles_system)
    
    cache_info = sne_cache_df.query('twinkles_system == %i' % twinkles_system)
    gal_id = cache_info['galtileid'].values[0]
    lens_info = sne_df.iloc[np.where(sne_df['twinkles_sysno'] == twinkles_system)]
    bulge_info = df_galaxy.query('galaxy_id == %i' % gal_id)
    disk_info = df_disk.query('galaxy_id == %i' % gal_id)

    
    #print(bulge_info, disk_info)
    
    for lens_image in lens_info['imno'].values:
        ra_shift = np.radians(lens_info['x'].iloc[lens_image] / 3600.) / np.cos(np.radians(bulge_info['decPhoSim']))
        dec_shift = np.radians(lens_info['y'].iloc[lens_image] / 3600.)
        
        image_ra = np.radians(bulge_info['raPhoSim']) + ra_shift
        image_dec = np.radians(bulge_info['decPhoSim'] + dec_shift)
        
        ra_images.append(np.degrees(image_ra).values[0])
        dec_images.append(np.degrees(image_dec).values[0])
        num_images.append(lens_image)
        unique_id_lens.append(bulge_info['uniqueId'].values[0])
        twinkles_system_images.append(twinkles_system)
        base_id = (gal_id + int(1.5e10))*10000 + twinkles_system*4 + lens_image
        unique_id_bulge.append(np.left_shift(base_id, 10) + 97)
        unique_id_disk.append(np.left_shift(base_id, 10) + 107)
        
        bulge_index.append(bulge_info.index[0])
        
        if len(disk_info) == 0:
            print(twinkles_system)
        else:
            disk_index.append(disk_info.index[0])
            disk_valid.append(i)
            
        i += 1

1100
1200
1300
1400
1500
1600
1700
1800
1900
2000


In [34]:
host_bulge = df_galaxy.iloc[bulge_index]

In [35]:
host_bulge = host_bulge.reset_index(drop=True)

In [36]:
host_bulge['twinkles_system'] = twinkles_system_images

In [37]:
host_bulge['image_number'] = num_images

In [38]:
host_bulge['uniqueId_lens'] = unique_id_lens

In [39]:
host_bulge['uniqueId'] = unique_id_bulge

In [40]:
host_bulge['raPhoSim_lens'] = host_bulge['raPhoSim']
host_bulge['decPhoSim_lens'] = host_bulge['decPhoSim']

In [41]:
host_bulge['raPhoSim'] = ra_images
host_bulge['decPhoSim'] = dec_images

In [42]:
host_bulge.head()

,prefix,uniqueId,raPhoSim,decPhoSim,phosimMagNorm,sedFilepath,redshift,shear1,shear2,kappa,...,internalRv,galacticExtinctionModel,galacticAv,galacticRv,galaxy_id,twinkles_system,image_number,uniqueId_lens,raPhoSim_lens,decPhoSim_lens
0,object,163841714682265697,52.956113,-27.626196,25.461545,galaxySED/Inst.62E09.04Z.spec.gz,0.178639,-0.002919,-0.000782,-0.000614,...,0.495764,CCM,0.025827,3.1,1000167449,1100,0,1024171467873,52.955893,-27.626193
1,object,163841714682266721,52.955796,-27.626191,25.461545,galaxySED/Inst.62E09.04Z.spec.gz,0.178639,-0.002919,-0.000782,-0.000614,...,0.495764,CCM,0.025827,3.1,1000167449,1100,1,1024171467873,52.955893,-27.626193
2,object,163846887991709793,52.901059,-27.672699,25.510829,galaxySED/Burst.80E09.04Z.spec.gz,0.241535,-0.003348,0.000128,-0.001160,...,0.460691,CCM,0.024221,3.1,1000672655,1101,0,1024688798817,52.901115,-27.672702
3,object,163846887991710817,52.901237,-27.672701,25.510829,galaxySED/Burst.80E09.04Z.spec.gz,0.241535,-0.003348,0.000128,-0.001160,...,0.460691,CCM,0.024221,3.1,1000672655,1101,1,1024688798817,52.901115,-27.672702
4,object,163846887991711841,52.901066,-27.672703,25.510829,galaxySED/Burst.80E09.04Z.spec.gz,0.241535,-0.003348,0.000128,-0.001160,...,0.460691,CCM,0.024221,3.1,1000672655,1101,2,1024688798817,52.901115,-27.672702


In [43]:
host_disk = df_disk.iloc[disk_index]

In [44]:
host_disk = host_disk.reset_index(drop=True)

In [45]:
host_disk['twinkles_system'] = np.array(twinkles_system_images)[disk_valid]

In [46]:
host_disk['image_number'] = np.array(num_images)[disk_valid]

In [47]:
host_disk['uniqueId_lens'] = np.array(unique_id_lens)[disk_valid]

In [48]:
host_disk['uniqueId'] = np.array(unique_id_disk)[disk_valid]

In [49]:
host_disk['raPhoSim_lens'] = host_disk['raPhoSim']
host_disk['decPhoSim_lens'] = host_disk['decPhoSim']

In [50]:
host_disk['raPhoSim'] = np.array(ra_images)[disk_valid]
host_disk['decPhoSim'] = np.array(dec_images)[disk_valid]

In [51]:
host_disk.head()

,prefix,uniqueId,raPhoSim,decPhoSim,phosimMagNorm,sedFilepath,redshift,shear1,shear2,kappa,...,internalRv,galacticExtinctionModel,galacticAv,galacticRv,galaxy_id,twinkles_system,image_number,uniqueId_lens,raPhoSim_lens,decPhoSim_lens
0,object,163841714682265707,52.956113,-27.626196,23.902496,galaxySED/Inst.50E09.02Z.spec.gz,0.178639,-0.002919,-0.000782,-0.000614,...,0.438248,CCM,0.025827,3.1,1000167449,1100,0,1024171467873,52.955893,-27.626193
1,object,163841714682266731,52.955796,-27.626191,23.902496,galaxySED/Inst.50E09.02Z.spec.gz,0.178639,-0.002919,-0.000782,-0.000614,...,0.438248,CCM,0.025827,3.1,1000167449,1100,1,1024171467873,52.955893,-27.626193
2,object,163846887991709803,52.901059,-27.672699,24.289415,galaxySED/Burst.62E09.02Z.spec.gz,0.241535,-0.003348,0.000128,-0.001160,...,0.439848,CCM,0.024221,3.1,1000672655,1101,0,1024688798817,52.901115,-27.672702
3,object,163846887991710827,52.901237,-27.672701,24.289415,galaxySED/Burst.62E09.02Z.spec.gz,0.241535,-0.003348,0.000128,-0.001160,...,0.439848,CCM,0.024221,3.1,1000672655,1101,1,1024688798817,52.901115,-27.672702
4,object,163846887991711851,52.901066,-27.672703,24.289415,galaxySED/Burst.62E09.02Z.spec.gz,0.241535,-0.003348,0.000128,-0.001160,...,0.439848,CCM,0.024221,3.1,1000672655,1101,2,1024688798817,52.901115,-27.672702


In [53]:
host_bulge.to_csv('../../data/cosmoDC2_v1.0_bulge_sne_host.csv')
host_disk.to_csv('../../data/cosmoDC2_v1.0_disk_sne_host.csv')